based on: https://www.tensorflow.org/tutorials/keras/text_classification_with_hub

In [ ]:
!pip3 install -q tfds-nightly
!pip3 install -q tensorflow-hub
!pip3 install mlflow

     |████████████████████████████████| 13.9MB 241kB/s 
     |████████████████████████████████| 153kB 41.0MB/s 
     |████████████████████████████████| 6.0MB 43.3MB/s 
     |████████████████████████████████| 81kB 9.5MB/s 
     |████████████████████████████████| 1.1MB 43.3MB/s 
     |████████████████████████████████| 327kB 38.6MB/s 
     |████████████████████████████████| 163kB 40.4MB/s 
     |████████████████████████████████| 61kB 7.9MB/s 
     |████████████████████████████████| 204kB 42.9MB/s 
     |████████████████████████████████| 81kB 7.3MB/s 
     |████████████████████████████████| 2.6MB 42.9MB/s 
     |████████████████████████████████| 122kB 43.3MB/s 
     |████████████████████████████████| 92kB 10.3MB/s 
     |████████████████████████████████| 71kB 8.9MB/s 
     |████████████████████████████████| 51kB 6.9MB/s 
  Created wheel for sqlalchemy: filename=SQLAlchemy-1.3.13-cp36-cp36m-linux_x86_64.whl size=1217160 sha256=da90bd2f08b47c47bb55a27678da9f87abb66d30e23e62f6c5cf410b8f29a7ba

In [ ]:
import os
import numpy as np
import mlflow

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

# Load compressed models from tensorflow_hub
os.environ["TFHUB_MODEL_LOAD_FORMAT"] = "COMPRESSED"

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")

Version:  2.3.0
Eager mode:  True
Hub version:  0.9.0
GPU is available


In [ ]:
!pip3 freeze > /content/requirements.txt

In [ ]:
#config 

NGROK_URL='http://ec2-3-133-154-208.us-east-2.compute.amazonaws.com:5001/'
EXPERIMENT='nlp/tfhub/IMDB_movie'
EPOCHS=10
VERSION=1
SAVE_PATH='/tmp/mlflorunstore'

loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
optimizer= 'adam'

%mkdir -p SAVE_PATH

In [ ]:
#Mlflow settings

#set MLflow server 
mlflow.set_tracking_uri(NGROK_URL)
#Set experiment
if mlflow.get_experiment_by_name(EXPERIMENT) != None:
    exp_id = mlflow.set_experiment(EXPERIMENT)
else: 
    exp_id = mlflow.create_experiment(EXPERIMENT)

#Close active runs
if mlflow.active_run():
    mlflow.end_run()

In [ ]:
# Download the IMDB dataset
# Split the training set into 60% and 40%, so we'll end up with 15,000 examples
# for training, 10,000 examples for validation and 25,000 examples for testing.
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews", 
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteOXI52S/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteOXI52S/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteOXI52S/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [ ]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

In [ ]:
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0])>

One way to represent the text is to convert sentences into embeddings vectors. 
We can use a pre-trained text embedding as the first layer, which will have three advantages:

* we don't have to worry about text preprocessing,
* we can benefit from transfer learning,
* the embedding has a fixed size, so it's simpler to process.

For this example we will use a pre-trained text embedding model from **TensorFlow Hub** called **google/nnlm-en-dim50/2** 

For info on [embedding](https://www.tensorflow.org/tutorials/text/word_embeddings).

In [ ]:
#Let's first create a Keras layer that uses a TensorFlow Hub model to embed the sentences, and try it out on a couple of input examples. 
#Note that no matter the length of the input text, the output shape of the embeddings is: (num_examples, embedding_dimension).

embedding = "https://tfhub.dev/google/nnlm-en-dim50/2"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

<tf.Tensor: shape=(3, 50), dtype=float32, numpy=
array([[ 0.5423195 , -0.0119017 ,  0.06337538,  0.06862972, -0.16776837,
        -0.10581174,  0.16865303, -0.04998824, -0.31148055,  0.07910346,
         0.15442263,  0.01488662,  0.03930153,  0.19772711, -0.12215476,
        -0.04120981, -0.2704109 , -0.21922152,  0.26517662, -0.80739075,
         0.25833532, -0.3100421 ,  0.28683215,  0.1943387 , -0.29036492,
         0.03862849, -0.7844411 , -0.0479324 ,  0.4110299 , -0.36388892,
        -0.58034706,  0.30269456,  0.3630897 , -0.15227164, -0.44391504,
         0.19462997,  0.19528408,  0.05666234,  0.2890704 , -0.28468323,
        -0.00531206,  0.0571938 , -0.3201318 , -0.04418665, -0.08550783,
        -0.55847436, -0.23336391, -0.20782952, -0.03543064, -0.17533456],
       [ 0.56338924, -0.12339553, -0.10862679,  0.7753425 , -0.07667089,
        -0.15752277,  0.01872335, -0.08169781, -0.3521876 ,  0.4637341 ,
        -0.08492756,  0.07166859, -0.00670817,  0.12686075, -0.19326553,
 

In [ ]:
#Let's now build the full model:

model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 50)                48190600  
_________________________________________________________________
dense (Dense)                (None, 16)                816       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 48,191,433
Trainable params: 48,191,433
Non-trainable params: 0
_________________________________________________________________


The layers are stacked sequentially to build the classifier:

1. The first layer is a TensorFlow Hub layer. This layer uses a pre-trained Saved Model to map a sentence into its **embedding vector**. The pre-trained text embedding model that we are using (google/nnlm-en-dim50/2) splits the sentence into tokens, embeds each token and then combines the embedding. The resulting dimensions are: (num_examples, embedding_dimension). For this NNLM model, the embedding_dimension is 50.

2. This fixed-length output vector is piped through a fully-connected (Dense) layer with 16 hidden units.

3. The last layer is densely connected with a single output node.

In [ ]:
#Since this is a binary classification problem and the model outputs logits 
#(a single-unit layer with a linear activation), we'll use the binary_crossentropy loss function.

#This isn't the only choice for a loss function, you could, for instance, choose mean_squared_error. 
#But, generally, binary_crossentropy is better for dealing with probabilities—it measures the "distance" 
#between probability distributions, or in our case, between the ground-truth distribution and the predictions.



model.compile(optimizer=optimizer,
              loss=loss,
              metrics=['accuracy'])

In [ ]:
with mlflow.start_run(run_id=None, experiment_id=exp_id, run_name=None, nested=False): 
  history = model.fit(train_data.shuffle(10000).batch(512),
                      epochs=EPOCHS,
                      validation_data=validation_data.batch(512),
                      verbose=1)
  
  hist=history.history

  #mlflow logging
      
  # log parameters
  mlflow.log_param("epochs", EPOCHS)
  mlflow.log_param("loss_function", str(loss))
  mlflow.log_param("optimizer", 'optimizer')

  # log metrics
  for epoch in range(0, EPOCHS):
    mlflow.log_metric("accuracy", hist['accuracy'][epoch])
    mlflow.log_metric("loss",  hist['loss'][epoch])
    mlflow.log_metric("val_accuracy",  hist['val_accuracy'][epoch])
    mlflow.log_metric("val_loss",  hist['val_loss'][epoch])
  
  #log model
  #model.save(os.path.join(BASE_DIR, "models", "{}.h5".format(int(t)))) #HDF5 format
  tf.saved_model.save(model, SAVE_PATH) #SavedModel format
  #mlflow.tensorflow.log_model(model, 'model') #TODO fix
  
  # log artifacts (matplotlib images for loss/accuracy)
  #mlflow.log_artifacts(model_folder)

  mlflow.end_run()

Epoch 1/10
30/30 [==============================] - 4s 122ms/step - loss: 0.6449 - accuracy: 0.5409 - val_loss: 0.5891 - val_accuracy: 0.6499
Epoch 2/10
30/30 [==============================] - 3s 115ms/step - loss: 0.5130 - accuracy: 0.7215 - val_loss: 0.4706 - val_accuracy: 0.7576
Epoch 3/10
30/30 [==============================] - 3s 111ms/step - loss: 0.3765 - accuracy: 0.8370 - val_loss: 0.3824 - val_accuracy: 0.8364
Epoch 4/10
30/30 [==============================] - 3s 112ms/step - loss: 0.2693 - accuracy: 0.8973 - val_loss: 0.3351 - val_accuracy: 0.8462
Epoch 5/10
30/30 [==============================] - 3s 114ms/step - loss: 0.1954 - accuracy: 0.9307 - val_loss: 0.3138 - val_accuracy: 0.8640
Epoch 6/10
30/30 [==============================] - 3s 111ms/step - loss: 0.1428 - accuracy: 0.9557 - val_loss: 0.3069 - val_accuracy: 0.8680
Epoch 7/10
30/30 [==============================] - 3s 113ms/step - loss: 0.1030 - accuracy: 0.9717 - val_loss: 0.3104 - val_accuracy: 0.8725
Epoch 

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


INFO:tensorflow:Assets written to: /tmp/mlflorunstore/assets


INFO:tensorflow:Assets written to: /tmp/mlflorunstore/assets


In [ ]:
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

49/49 - 2s - loss: 0.3676 - accuracy: 0.8502
loss: 0.368
accuracy: 0.850
